In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from utils import get_files, get_labels, show_conditions, get_data, get_working_data, get_peak_trace, show_peaks

In [ ]:
from scipy.signal import savgol_filter

# 1 - Wczytaj pliki

In [ ]:
dfs = get_files()

In [ ]:
i = 12 # numer tabeli

In [ ]:
dfa = dfs[i]
dfa

In [ ]:
df = dfa.copy()
#len(df)
#df
print(dfa.loc[[4001]])

In [ ]:
df = df.loc[16000:134500]
df = df.reset_index(level=None, drop=True)
df

In [ ]:
labels_cond, labels_i = get_labels(df)

# 2 - Zaznacz warunki

In [ ]:
#labels_cond, labels_i = get_labels(df)
df['Warunek'] = labels_cond
df['Interwał'] = labels_i

In [ ]:
df_example = df.groupby(['Warunek', 'Interwał']).sum().reset_index()
df_example = df_example[['Warunek', 'Interwał']]
df_example[df_example['Warunek'] != 'brak']

In [ ]:
show_conditions(start=0, end=156000, df=df)

# 3 - Licz piki
1. Zobacz wyczyszczony sygnał.
2. Podaj parametry.
3. Zobacz wynik na wykresie.
4. Jeśli wynik się nie pdoba, to zmień wartość parametrów.

## Pokaż wyczyszczony sygnał

In [ ]:
fig = px.line(df, x = df.index, y = 'CleanSignal',title = 'Dane przed transaformacją')
fig.show()

In [ ]:
df.CleanSignal = savgol_filter(df.CleanSignal, 17, 2) #filtruje sygnał tak by był bardziej smooth

In [ ]:
fig = px.line(df, x = df.index, y = 'CleanSignal',title = 'Dane po smoothowaniu')
fig.show()

## Parametry
### Filtrowanie danych:
- *max_good_value* - największa rzetelna wartośc sygnału
- *min_good_value* - najmniejsza rzetelna wartośc sygnału

### Skalowanie danych: spąłszcenie do podanego zakresu
- *upper* - górna granica nowego zakresu, domyslnie 1024
- *lower* - dolna granica nowego zakresu, domyślnie 0

### Inne
- *hz* - częstotliwosć pomiaru, domyślnie 100
- *col* - kolumna z sygnałem, domyślnie 'CleanSignal' 

In [ ]:
df['CleanSignal'].max()

In [ ]:
df['CleanSignal'].min()

In [ ]:
max_good_value = df['CleanSignal'].max()
min_good_value = df['CleanSignal'].min()
upper = 1024 #domyslnie 1024
lower = 0 # domyślnie 0
hz = 100 # domyślnie 100
col = 'CleanSignal' # domyślnie 'CleanSignal' 
df_len = len(df)
#hrw = 0.75 #wielkosc okna do probkowania dla sredniej kroczacej

In [ ]:
data = get_data(df.reset_index(), col, max_good_value, min_good_value, lower, upper)
working_data = get_working_data(data, hz)
peak_trace = get_peak_trace(working_data, df_len)
df['Peak'] = peak_trace

In [ ]:
#co liczymy? suma pików dla interwałów
df_grouped = df.groupby(['Warunek', 'Interwał']).agg({'Peak': 'sum'}).reset_index()
df_grouped = df_grouped[df_grouped['Warunek'] != 'brak']
print(f'Wszystkie piki: {df["Peak"].sum()}')
df_grouped

In [ ]:
show_peaks(df)

# 4 - Zapisz dane
Wpisz odpowiednie `person_id` i przeklikaj resztę:
- sprawdź czy nowa tabela `new_df` jest ok
- sprwadź czy dobrze się wcyztaął tabela ze  wszystkimi wynikami `full_df`
- przejdź przez skrypt dla następnej osoby :)

In [ ]:
person_id = 'P004' #zmien osobe

In [ ]:
new_df = df_grouped.copy()
new_df['id'] = person_id
new_df

In [ ]:
new_df.to_excel(f'./peak_data/{person_id}.xlsx')